# Adama example for DesignSafe-CI

This is an example of building an Adama service.  We use the Haiti Earthquake Database and we construct a couple of web services from the data hosted at https://nees.org/dataview/spreadsheet/haiti.

## Setting up

The code for these services is in the directory `demo`:

In [12]:
cd demo

/data/local/demo


We will construct two web services that will return the data in JSON format:

- `haiti`: will allow to query the database by building,
- `haiti_images`: will allow to retrieve the set of images for each building.

Each Adama service consists in two pieces of information:

- the metadata that describes the service,
- and the actual code.

This is an example of the metadata for the `haiti` service:

In [ ]:
%load services/haiti/metadata.yml

The code for the service looks very simple:

In [ ]:
%%writefile services/haiti/main.py
import json

import services.common.tools as t


def search(args, adama):
    """Search database by building id."""

    building_id = args['building']
    for row in t.HAITI_DB:
        if row['Building'].startswith(building_id):
            print json.dumps(row, indent=4)
            print '---'
        

def list(args, adama):
    """List all rows."""

    for row in t.HAITI_DB:
        print json.dumps(row, indent=4)
        print '---'


## Interacting with Adama

To interact with Adama, we create an object with our credentials:

In [5]:
import adamalib

adama = adamalib.Adama('https://api.araport.org/community/v0.3',   # we use Araport for this example
                       token='474af9d41c8ecc873191ea97153857')                              # here goes your credentials

Now the `adama` object is connected to the server.  We can check that the server is up:

In [6]:
adama.status

{u'api': u'Adama v0.3',
 u'hash': u'1f5e292b0df113c7e36bad18ad0f11184610e187',
 u'status': u'success'}

## Testing the new services locally

The services we are going to register in Adama can be tested first locally:

In [11]:
import services.haiti.main

services.haiti.main.search({'building': 'A001'}, adama)

{
    "Effective  Wall Area": "0", 
    "Captive  Columns": "Y", 
    "Priority Index  [%]": "0.17", 
    "Pictures": "https://nees.org/site/collections/haiti/mainpics/A001.JPG", 
    "Masonry Wall  Area E-W [ft\u00b2]": "102", 
    "Latitude": "18\u00b0 33' 16.1\" N", 
    "Concrete Wall  Area N-S [ft\u00b2]": "0", 
    "Total Floor Area  [ft\u00b2]": "19321", 
    "First Floor Area  [ft\u00b2]": "9661", 
    "Diagram": "https://nees.org/site/collections/haiti/Diagrams/A001-Diagram.pdf", 
    "Concrete Wall  Area E-W [ft\u00b2]": "0", 
    "Photographer": "Steeve", 
    "Date": "26-Jun", 
    "Building": "A001", 
    "Column Index  [%]": "0.17", 
    "Masonry Wall  Damage": "Severe", 
    "Notes": "Pic. 1544 for arch. Detail on E. side/ ~2\" mortar cover on bldg. cols.", 
    "Permenant  Drift": "N", 
    "Ace  [ft\u00b2]": "33", 
    "Longitude": "72\u00b0 18' 12.1\" W", 
    "Roof  Type": "Concrete", 
    "Masonry Wall  Area N-S [ft\u00b2]": "0", 
    "Wall Index  [%]": "0", 
    "T

## Registering the services in Adama

Now we are ready to register the services in Adama.  We'll use the namespace `walter-dev` for testing purposes:

In [12]:
adama['walter-dev'].services

[Service(/walter-dev/araport11_to_json_v0.1),
 Service(/walter-dev/smallrna_v0.1),
 Service(/walter-dev/haiti_images_v0.1),
 Service(/walter-dev/strandinfo_v0.5),
 Service(/walter-dev/at_abundances_v0.1),
 Service(/walter-dev/expression_per_gene_tissue_01_v0.1),
 Service(/walter-dev/at_srna_docs_v0.1),
 Service(/walter-dev/gene-api_main_v0.1),
 Service(/walter-dev/atted_coexpressed_by_locus_v0.2.0),
 Service(/walter-dev/smallrna_v2),
 Service(/walter-dev/expression_condition_comparison_v0.1),
 Service(/walter-dev/haiti_v0.1),
 Service(/walter-dev/expression_per_gene_tissue_04_v0.1)]

To register a service, we just import its code and add it to the previous list (it may take a minute or two):

In [ ]:
import services.haiti.main            # the haiti service
import services.haiti_images.main     # the haiti_images service

haiti = adama['walter-dev'].services.add(services.haiti.main)
haiti_images = adama['walter-dev'].services.add(services.haiti_images.main)

haiti, haiti_images

The services are registered and can be accessed in https://araport.org.

But they can also be tested using the service object via Python.  Note that now the code will be executed remotely in the Adama server:

In [ ]:
haiti.search(building='A001')

In [ ]:
from IPython.display import Image

Image(data=haiti_images.search(building='A001', image=1).content)

## Using the services

Now that the services are registered in Adama, and the Python objects `haiti` and `haiti_images` are connected to the **remote** services, we can use them as regular objects.

In [19]:
!pip install pandas

    100% |████████████████████████████████| 4.9MB 137kB/s 
    100% |████████████████████████████████| 475kB 1.1MB/s 
    100% |████████████████████████████████| 4.0MB 165kB/s 
  Running setup.py bdist_wheel for pandas
  Stored in directory: /root/.cache/pip/wheels/e5/1e/7a/d077ee541ea1e5a58df866719f9d837d5921afb4e9bf9b9e8d
  Running setup.py bdist_wheel for numpy
  Stored in directory: /root/.cache/pip/wheels/4b/cb/75/2cf74e8d40c2f6e466c621d598769f961bbb0feac948013dfb
Successfully built pandas numpy


In [21]:
from pandas import DataFrame
x = DataFrame([[1,2], [2,3]], index=['Foo', 'Bar'], columns=['ds','sf'])

In [7]:
haiti = adama['walter-dev'].haiti
haiti_images = adama['walter-dev'].haiti_images

Let's display the table of buildings together with their geographical coordinates:

In [29]:
full = haiti.list()
columns = ['Building', 'Latitude', 'Longitude']
DataFrame([[row[col] for col in columns] for row in full],
          columns=columns)

,Building,Latitude,Longitude
0,A001,18.55447,-72.30336
1,A002,18.55436,-72.30297
2,A003(1),18.51167,-72.28739
3,A003(2),18.51167,-72.28739
4,A005,18.51811,-72.28222
5,A006,18.54864,-72.30319
6,A007,18.54892,-72.30386
7,A008,18.54881,-72.30683
8,A009,,
9,A010,18.55094,-72.30825


In [30]:
import jinja2

In [43]:
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets
from IPython.display import clear_output, display, HTML
import requests

In [52]:
!pip install gmaps

    100% |████████████████████████████████| 102kB 3.4MB/s 
  Running setup.py bdist_wheel for gmaps
  Stored in directory: /root/.cache/pip/wheels/fa/aa/18/3bb6bac4a63612f6bb563d781cac880b9d573cdf4a3c28437d
Successfully built gmaps


In [53]:
import gmaps

<IPython.core.display.Javascript object>

In [54]:
data = gmaps.datasets.load_dataset('taxi_rides')

In [64]:
gmaps.

array([(37.782551, -122.445368), (37.782745, -122.444586),
       (37.782842, -122.443688), (37.782919, -122.442815),
       (37.782992, -122.442112), (37.7831, -122.441461),
       (37.783206, -122.440829), (37.783273, -122.440324),
       (37.783316, -122.440023), (37.783357, -122.439794),
       (37.783371, -122.439687), (37.783368, -122.439666),
       (37.783383, -122.439594), (37.783508, -122.439525),
       (37.783842, -122.439591), (37.784147, -122.439668),
       (37.784206, -122.439686), (37.784386, -122.43979),
       (37.784701, -122.439902), (37.784965, -122.439938),
       (37.78501, -122.439947), (37.78536, -122.439952),
       (37.785715, -122.44003), (37.786117, -122.440119),
       (37.786564, -122.440209), (37.786905, -122.44027),
       (37.786956, -122.440279), (37.800224, -122.43352),
       (37.800155, -122.434101), (37.80016, -122.43443),
       (37.800378, -122.434527), (37.800738, -122.434598),
       (37.800938, -122.43465), (37.801024, -122.434889),
       (